In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, confusion_matrix, classification_report
from statsmodels.api import Logit

In [2]:
df = pd.read_csv("../data/cleaned_data.csv")

In [3]:
df.head()

,TransactionId,BatchId,AccountId,SubscriptionId,CustomerId,CurrencyCode,CountryCode,ProviderId,ProductId,ProductCategory,ChannelId,Amount,Value,TransactionStartTime,PricingStrategy,FraudResult
0,76871,36123,3957,887,4406,UGX,256,6,10,airtime,3,1000.0,1000,2018-11-15 02:18:49+00:00,2,0
1,73770,15642,4841,3829,4406,UGX,256,4,6,financial_services,2,-20.0,20,2018-11-15 02:19:08+00:00,2,0
2,26203,53941,4229,222,4683,UGX,256,6,1,airtime,3,500.0,500,2018-11-15 02:44:21+00:00,2,0
3,380,102363,648,2185,988,UGX,256,1,21,utility_bill,3,20000.0,21800,2018-11-15 03:32:55+00:00,2,0
4,28195,38780,4841,3829,988,UGX,256,4,6,financial_services,2,-644.0,644,2018-11-15 03:34:21+00:00,2,0


Feature engineering

In [4]:
# Calculate the aggregate features
aggregate_features = df.groupby('CustomerId').agg(
    total_transaction_amount=pd.NamedAgg(column='Amount', aggfunc='sum'),
    average_transaction_amount=pd.NamedAgg(column='Amount', aggfunc='mean'),
    transaction_count=pd.NamedAgg(column='TransactionId', aggfunc='count'),
    stddev_transaction_amount=pd.NamedAgg(column='Amount', aggfunc='std')
).reset_index()


aggregate_features

,CustomerId,total_transaction_amount,average_transaction_amount,transaction_count,stddev_transaction_amount
0,1,-10000.0,-10000.00000,1,NaN
1,2,-10000.0,-10000.00000,1,NaN
2,3,-10000.0,-10000.00000,1,NaN
3,4,-10000.0,-10000.00000,1,NaN
4,5,-10000.0,-10000.00000,1,NaN
...,...,...,...,...,...
3737,7475,-6000.0,-2000.00000,3,2179.449472
3738,7476,-213400.0,-9278.26087,23,9563.137395
3739,7477,-2000.0,-2000.00000,1,NaN
3740,7478,500.0,500.00000,1,NaN


In [6]:
df['TransactionStartTime'] = pd.to_datetime(df['TransactionStartTime'])
# Extract the features
df['TransactionYear'] = df['TransactionStartTime'].dt.year
df['TransactionMonth'] = df['TransactionStartTime'].dt.month
df['TransactionDay'] = df['TransactionStartTime'].dt.day
df['TransactionHour'] = df['TransactionStartTime'].dt.hour

In [7]:
df.head()

,TransactionId,BatchId,AccountId,SubscriptionId,CustomerId,CurrencyCode,CountryCode,ProviderId,ProductId,ProductCategory,ChannelId,Amount,Value,TransactionStartTime,PricingStrategy,FraudResult,TransactionYear,TransactionMonth,TransactionDay,TransactionHour
0,76871,36123,3957,887,4406,UGX,256,6,10,airtime,3,1000.0,1000,2018-11-15 02:18:49+00:00,2,0,2018,11,15,2
1,73770,15642,4841,3829,4406,UGX,256,4,6,financial_services,2,-20.0,20,2018-11-15 02:19:08+00:00,2,0,2018,11,15,2
2,26203,53941,4229,222,4683,UGX,256,6,1,airtime,3,500.0,500,2018-11-15 02:44:21+00:00,2,0,2018,11,15,2
3,380,102363,648,2185,988,UGX,256,1,21,utility_bill,3,20000.0,21800,2018-11-15 03:32:55+00:00,2,0,2018,11,15,3
4,28195,38780,4841,3829,988,UGX,256,4,6,financial_services,2,-644.0,644,2018-11-15 03:34:21+00:00,2,0,2018,11,15,3


In [8]:
categorical_columns = ['CurrencyCode', 'ProductCategory']

# Create a OneHotEncoder object
one_hot_encoder = OneHotEncoder(sparse_output=False, drop='first')  # drop='first' to avoid dummy variable trap

# Fit and transform the data
one_hot_encoded = one_hot_encoder.fit_transform(df[categorical_columns])

one_hot_encoded_df = pd.DataFrame(one_hot_encoded, columns=one_hot_encoder.get_feature_names_out(categorical_columns))

df = pd.concat([df.drop(columns=categorical_columns), one_hot_encoded_df], axis=1)

df.head()

,TransactionId,BatchId,AccountId,SubscriptionId,CustomerId,CountryCode,ProviderId,ProductId,ChannelId,Amount,...,TransactionDay,TransactionHour,ProductCategory_data_bundles,ProductCategory_financial_services,ProductCategory_movies,ProductCategory_other,ProductCategory_ticket,ProductCategory_transport,ProductCategory_tv,ProductCategory_utility_bill
0,76871,36123,3957,887,4406,256,6,10,3,1000.0,...,15,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,73770,15642,4841,3829,4406,256,4,6,2,-20.0,...,15,2,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,26203,53941,4229,222,4683,256,6,1,3,500.0,...,15,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,380,102363,648,2185,988,256,1,21,3,20000.0,...,15,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,28195,38780,4841,3829,988,256,4,6,2,-644.0,...,15,3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
# Identify the numerical columns to be normalized
numerical_columns = ['Amount', 'Value']

# Create a MinMaxScaler object
min_max_scaler = MinMaxScaler()

# Fit and transform the data
df[numerical_columns] = min_max_scaler.fit_transform(df[numerical_columns])


df.head()

,TransactionId,BatchId,AccountId,SubscriptionId,CustomerId,CountryCode,ProviderId,ProductId,ChannelId,Amount,...,TransactionDay,TransactionHour,ProductCategory_data_bundles,ProductCategory_financial_services,ProductCategory_movies,ProductCategory_other,ProductCategory_ticket,ProductCategory_transport,ProductCategory_tv,ProductCategory_utility_bill
0,76871,36123,3957,887,4406,256,6,10,3,0.092004,...,15,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,73770,15642,4841,3829,4406,256,4,6,2,0.091910,...,15,2,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,26203,53941,4229,222,4683,256,6,1,3,0.091958,...,15,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,380,102363,648,2185,988,256,1,21,3,0.093750,...,15,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,28195,38780,4841,3829,988,256,4,6,2,0.091853,...,15,3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
# Identify the numerical columns to be standardized
numerical_columns = ['Amount', 'Value']

# Create a StandardScaler object
standard_scaler = StandardScaler()

# Fit and transform the data
df[numerical_columns] = standard_scaler.fit_transform(df[numerical_columns])

df.head()

,TransactionId,BatchId,AccountId,SubscriptionId,CustomerId,CountryCode,ProviderId,ProductId,ChannelId,Amount,...,TransactionDay,TransactionHour,ProductCategory_data_bundles,ProductCategory_financial_services,ProductCategory_movies,ProductCategory_other,ProductCategory_ticket,ProductCategory_transport,ProductCategory_tv,ProductCategory_utility_bill
0,76871,36123,3957,887,4406,256,6,10,3,-0.046371,...,15,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,73770,15642,4841,3829,4406,256,4,6,2,-0.054643,...,15,2,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,26203,53941,4229,222,4683,256,6,1,3,-0.050426,...,15,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,380,102363,648,2185,988,256,1,21,3,0.107717,...,15,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,28195,38780,4841,3829,988,256,4,6,2,-0.059704,...,15,3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
df.to_csv("../data/data_with_new_features.csv", index=False)